In [3]:
import pymaid
url = "https://neurophyla.mrc-lmb.cam.ac.uk/catmaid/fibsem/#"
token = "4fe0c0572331265c61bf6ba4acd77cbed9792276"
name = "SYLee"
password = "Blue Skies"
project_id = 18
rm = pymaid.CatmaidInstance(url, token, name, password, project_id)

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


In [94]:
## import pandas as pd 
fp_df=pd.read_csv('~/Documents/CATMAID/pymaid/8_falsepositive/octo_cube3_same_preid_128/false_positives.csv')
gt_df=pd.read_csv('~/Documents/CATMAID/pymaid/8_falsepositive/octo_cube3_same_preid_128/matched_pairs.csv')

In [95]:
print(gt_df.columns)

Index(['pred_pre_x', 'pred_pre_y', 'pred_pre_z', 'pred_post_x', 'pred_post_y',
       'pred_post_z', 'gt_pre_x', 'gt_pre_y', 'gt_pre_z', 'gt_post_x',
       'gt_post_y', 'gt_post_z', 'distance'],
      dtype='object')


In [96]:
bbox3=pd.read_csv('synapsescube3_bbox.csv')
print(bbox3.columns)
#bbox in pixels 
x_max = int(bbox3['x_max'].iloc[0])
y_min = int(bbox3['y_min'].iloc[0])
z_min = int(bbox3['z_min'].iloc[0])

Index(['x_min', 'x_max', 'y_min', 'y_max', 'z_min', 'z_max', 'x_distance',
       'y_distance', 'z_distance', 'number of points inside'],
      dtype='object')


In [99]:
def reverse_coords(coords):
    coords = list(coords)
    original_x = (x_max - (coords[0]/ 8)) * 8
    original_y = (coords[1] / 8 + y_min) * 8
    original_z = (coords[2] / 8 + z_min) * 8
    return (original_x,original_y,original_z)

### ADD NODE(post)

In [100]:
import pandas as pd

# List to store results
results = []

for index, row in fp_df.iterrows():
    post_x, post_y, post_z = row["pred_post_x"], row["pred_post_y"], row["pred_post_z"]
    post_coords = (post_x, post_y, post_z)
    
    # Transform the coordinates
    post_coords_new = reverse_coords(post_coords)
    
    # Send the request and get response
    response = pymaid.add_node(post_coords_new, parent_id=None, radius=-1, confidence=5)

    # Store response data in a dictionary
    response_data = {
        "index": index,
        "pred_post_x": post_x,
        "pred_post_y": post_y,
        "pred_post_z": post_z,
        "post_x_new": post_coords_new[0],
        "post_y_new": post_coords_new[1],
        "post_z_new": post_coords_new[2],
        "treenode_id": response.get("treenode_id"),
        "skeleton_id": response.get("skeleton_id"),
        "edition_time": response.get("edition_time"),
        "parent_edition_time": response.get("parent_edition_time"),
        "created_links": response.get("created_links"),
    }

    # Append to results list
    results.append(response_data)

# Convert results to a new DataFrame
results_df = pd.DataFrame(results)

# Save results to a new CSV file
results_df.to_csv('~/Documents/CATMAID/pymaid/8_falsepositive/octo_cube3_same_preid_128/matched_pairs_with_nodes.csv', index=False)

print("Results saved to CSV successfully!")

Results saved to CSV successfully!


In [ ]:
bodyid=results_df['skeleton_id'].tolist()
pymaid.add_annotations(bodyid, 'pushed false positives synapses', remote_instance=None)

In [113]:
import pandas as pd

results = []  # Store results

for index, row in fp_df.iterrows():
    # Extract coordinates for pre and post
    pre_x, pre_y, pre_z = row["pred_pre_x"], row["pred_pre_y"], row["pred_pre_z"]
    post_x, post_y, post_z = row["pred_post_x"], row["pred_post_y"], row["pred_post_z"]
    
    pre_conn = (pre_x, pre_y, pre_z)
    post_conn = (post_x, post_y, post_z)
    
    # Transform pre-connection coordinates
    pre_conn_new = reverse_coords(pre_conn)
    pre_conn_new_list = list(pre_conn_new)

    # Push to CATMAID
    response = pymaid.add_connector(pre_conn_new_list, remote_instance=None)

    # Ensure response is a non-empty list before accessing it
    if response and isinstance(response, list):
        response_dict = response[0]  # Extract the first dictionary

        # Store response data
        response_data = {
            "index": index,
            "pred_pre_x": pre_x,
            "pred_pre_y": pre_y,
            "pred_pre_z": pre_z,
            "pred_post_x": post_x,  # Include post-synaptic coordinates
            "pred_post_y": post_y,
            "pred_post_z": post_z,
            "pre_x_new": pre_conn_new_list[0],
            "pre_y_new": pre_conn_new_list[1],
            "pre_z_new": pre_conn_new_list[2],
            "connector_id": response_dict.get("connector_id"),
            "connector_edition_time": response_dict.get("connector_edition_time"),
            "created_links": response_dict.get("created_links"),
        }

        # Append to results list
        results.append(response_data)

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save results to a CSV file
results_df.to_csv('~/Documents/CATMAID/pymaid/8_falsepositive/octo_cube3_same_preid_128/fp_connectors_push.csv', index=False)

print("Results saved to CSV successfully!")


Creating connectors: 0it [00:00, ?it/s]

Creating connectors: 0it [00:00, ?it/s]

Creating connectors: 0it [00:00, ?it/s]

Creating connectors: 0it [00:00, ?it/s]

Creating connectors: 0it [00:00, ?it/s]

Results saved to CSV successfully!


In [1]:
import pandas as pd

# Load the two CSV files
node = pd.read_csv("~/Documents/CATMAID/pymaid/8_falsepositive/octo_cube3_same_preid_128/matched_pairs_with_nodes.csv")  # Replace with your actual file path
connectors = pd.read_csv('~/Documents/CATMAID/pymaid/8_falsepositive/octo_cube3_same_preid_128/fp_connectors_push.csv')  # Replace with your actual file path

# Merge based on the overlapping columns
merged_df = pd.merge(node, connectors, on=["pred_post_x", "pred_post_y", "pred_post_z"], how="outer")


df = merged_df.drop(columns=[
    'index_x','edition_time', 'parent_edition_time',
    'created_links_x','connector_edition_time', 'created_links_y'])

print(df.columns)
df.to_csv("combination.csv", index=False)

print("Merged file saved as 'combination.csv'")

Index(['pred_post_x', 'pred_post_y', 'pred_post_z', 'post_x_new', 'post_y_new',
       'post_z_new', 'treenode_id', 'skeleton_id', 'index_y', 'pred_pre_x',
       'pred_pre_y', 'pred_pre_z', 'pre_x_new', 'pre_y_new', 'pre_z_new',
       'connector_id'],
      dtype='object')
Merged file saved as 'combination.csv'


In [ ]:
# Extract necessary columns and form a list of tuples
links = list(df[['treenode_id', 'connector_id']].itertuples(index=False, name=None))

# Add 'postsynaptic_to' as the third element in each tuple
links = [(node_id, conn_id, 'postsynaptic_to') for node_id, conn_id in links]

response = pymaid.link_connector(links)

# # Print response from the server
print(response)


In [64]:
for index , row in fp_df.iterrows():
    post_x, post_y, post_z = row["pred_post_x"],row["pred_post_y"],row["pred_post_z"]
    post_coords = (post_x, post_y, post_z)
    #pymaid.add_node((post_x, post_y, post_z), remote_instance=rm)
    post_coords_new = reverse_coords(post_coords)
    print(post_coords, post_coords_new)
    connectors = row[["pred_pre_x", "pred_pre_y", "pred_pre_z"]].tolist()
    conn_coords_new = reverse_coords(connectors)

    print(connectors, conn_coords_new)
    break
    #pymaid.add_connector()
    
#pymaid.add_node

(1368, 912, 184) ([1368, 912, 184], 0    48768.0
Name: x_max, dtype: float64, 0    26944.0
Name: y_min, dtype: float64, 0    59896.0
Name: z_min, dtype: float64)
[1336, 968, 112] ([1336, 968, 112], 0    48800.0
Name: x_max, dtype: float64, 0    27000.0
Name: y_min, dtype: float64, 0    59824.0
Name: z_min, dtype: float64)


In [57]:
for index , row in gt_df.iterrows():
    post_x, post_y, post_z = row["gt_post_x"],row["gt_post_y"],row["gt_post_z"]
    post_coords = (post_x, post_y, post_z)
    post_coords_new = reverse_coords(post_coords)
    pymaid.add_node((post_x, post_y, post_z), remote_instance=rm)
    # # print(post_coords, post_coords_new)
    # connectors = row[["gt_pre_x", "gt_pre_y", "gt_pre_z"]].tolist()
    # conn_coords_new = reverse_coords(connectors)

    # print(connectors, conn_coords_new)
    # if index > 2:
    break
    #pymaid.add_connector()
    
#pymaid.add_node

IndentationError: unexpected indent (1140428030.py, line 12)

In [ ]:
import pandas as pd

# Load the CSV containing skeleton IDs
post_push = pd.read_csv('~/Documents/CATMAID/pymaid/8_falsepositive/octo_cube3_same_preid_128/matched_pairs_with_nodes.csv')

# Loop through each row instead of using a list
for index, row in post_push.iterrows():
    skeleton_id = row["skeleton_id"]  # Extract skeleton ID
    
    if pd.isna(skeleton_id):  # Skip if skeleton_id is missing (NaN)
        print(f"Skipping row {index}: No skeleton_id found")
        continue
    
    try:
        # Attempt to delete neuron
        response = pymaid.delete_neuron(int(skeleton_id), no_prompt=True, remote_instance=None)
        print(f"Deleted skeleton_id {skeleton_id}: {response}")
    except Exception as e:
        # If deletion fails, print error and continue with the next row
        print(f"Failed to delete skeleton_id {skeleton_id}: {e}")


In [ ]:
neuron=pd.read_csv('combination.csv')
skids=neuron['skeleton_id'].tolist()
skids

In [7]:
import pandas as pd
import pymaid
import time

# Read the CSV
neuron = pd.read_csv('combination.csv')

# Extract skeleton IDs
skids = neuron['skeleton_id'].tolist()

# Function to split list into chunks of size 250
def chunk_list(lst, chunk_size=250):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

# Dictionary to store annotations
annotations_dict = {}

# Process in batches of 250
for batch in chunk_list(skids, 250):
    try:
        annotations = pymaid.get_annotations(batch)  # Fetch annotations
        annotations_dict.update(annotations)  # Merge results
    except Exception as e:
        print(f"Error fetching annotations for batch {batch}: {e}")
    time.sleep(1)  # Avoid overloading the server

# Convert dictionary to DataFrame
annotations_df = pd.DataFrame(list(annotations_dict.items()), columns=['skeleton_id', 'annotations'])

# Ensure skeleton_id type matches the original CSV
annotations_df['skeleton_id'] = annotations_df['skeleton_id'].astype(neuron['skeleton_id'].dtype)

# Expand the list of annotations into separate columns
annotations_expanded = annotations_df['annotations'].apply(pd.Series)

# Rename columns dynamically (Annotation_1, Annotation_2, etc.)
annotations_expanded.columns = [f'Annotation_{i+1}' for i in range(annotations_expanded.shape[1])]

# Combine skeleton_id with expanded annotations
annotations_df = pd.concat([annotations_df[['skeleton_id']], annotations_expanded], axis=1)

# Merge back with the original CSV
neuron = neuron.merge(annotations_df, on='skeleton_id', how='left')

# Save the updated DataFrame
neuron.to_csv('combination_updated.csv', index=False)

print("CSV updated successfully!")


INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


CSV updated successfully!
